In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.

In [ ]:
import pandas as pd
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments
import re

In [ ]:
# Load dataset
train_data = pd.read_csv("/content/samsum-train.csv")
validation_data = pd.read_csv("/content/samsum-validation.csv")

In [ ]:
train_data.head()

,id,dialogue,summary
0,13818513,Amanda: I baked cookies. Do you want some?\r\...,Amanda baked cookies and will bring Jerry some...
1,13728867,Olivia: Who are you voting for in this electio...,Olivia and Olivier are voting for liberals in ...
2,13681000,"Tim: Hi, what's up?\r\nKim: Bad mood tbh, I wa...",Kim may try the pomodoro technique recommended...
3,13730747,"Edward: Rachel, I think I'm in ove with Bella....",Edward thinks he is in love with Bella. Rachel...
4,13728094,Sam: hey overheard rick say something\r\nSam:...,"Sam is confused, because he overheard Rick com..."


In [ ]:
validation_data.head()

,id,dialogue,summary
0,13817023,"A: Hi Tom, are you busy tomorrow’s afternoon?\...",A will go to the animal shelter tomorrow to ge...
1,13716628,Emma: I’ve just fallen in love with this adven...,Emma and Rob love the advent calendar. Lauren ...
2,13829420,Jackie: Madison is pregnant\r\nJackie: but she...,Madison is pregnant but she doesn't want to ta...
3,13819648,Marla: <file_photo>\r\nMarla: look what I foun...,Marla found a pair of boxers under her bed.
4,13728448,Robert: Hey give me the address of this music ...,Robert wants Fred to send him the address of t...


In [ ]:
train_data = train_data.sample(n=4000,random_state=42).reset_index(drop=True)
validation_data = validation_data.sample(n=500, random_state=42).reset_index(drop=True)

In [ ]:
# Preprocessing
def clean_text(text):
    text = re.sub(r'\r\n', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'<.*?>', '', text)
    text = text.strip().lower()
    return text

train_data['dialogue'] = train_data['dialogue'].apply(clean_text)
train_data['summary'] = train_data['summary'].apply(clean_text)

validation_data['dialogue'] = validation_data['dialogue'].apply(clean_text)
validation_data['summary'] = validation_data['summary'].apply(clean_text)

In [ ]:
train_data

,id,dialogue,summary
0,13811908,violet: hi! i came across this austin's articl...,violet sent claire austin's article.
1,13716431,pat: so does anyone know when the stream is go...,pat and lou are waiting for the stream but kev...
2,13810214,jane: jane: whaddya think? shona: this ur tin...,jane is updating her tinder profile tonight an...
3,13729823,"adam: do u have a map of paris? tom: yes, why?...",tom has a map of paris.
4,13681400,"frank: hi, how's the family? mike: great! sam'...","mike is happy, because sam's moved out. mike a..."
...,...,...,...
3995,13681041,barry: hello buddy michael: hey barry: do you ...,barry and michael will watch football instead ...
3996,13818705,karen: hey lisa. larissa and me have recently ...,karen and larissa moved to belgium and ask lis...
3997,13821859,"miles: hey, guys, i'm so sorry, but i missed t...","miles has missed the bus, so he may be 15 minu..."
3998,13812716,emma: did you finish the book i gave you? liam...,"emma gave ""the first fifteen lives of harry au..."


In [ ]:
validation_data

,id,dialogue,summary
0,13680857,"edd: wow, did you hear that they're transferri...",rose and edd will be transferred to a new depa...
1,13716124,"tom: where is the ""sala del capitolo"" kevin: i...","""sala del capitolo"" tom is looking for is in t..."
2,13864418,patricia: the rowing practice is cancelled! ka...,the rowing practice is cancelled. a few member...
3,13729340,"tom: u ok? alex: yeah, pretty good. u? tom: a...",tom and alex had fun last night. they drank a ...
4,13818813,"patricia: hello, here's the fair-trade brand i...",patricia recommends a fair-trade brand she tal...
...,...,...,...
495,13820618,izzy: anyone knows where professor xavier has ...,professor xavier probably holds his duty hours...
496,13680766,"rita: didn't take breakfast with me, is there ...",lina will give rita one of her 2 sandwiches.
497,13819081,"peter: yo, we’re coming over to warsaw for thi...",peter and jen are coming to warsaw for the wee...
498,13862819,"francis: hey francis: listen, i need a favor. ...",francis asked reynold for help with installing...


In [ ]:
# Tokenization
tokenizer = T5Tokenizer.from_pretrained("t5-small")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
input_max_len = max(len(tokenizer.encode(text)) for text in train_data['dialogue'])
output_max_len = max(len(tokenizer.encode(text)) for text in train_data['summary'])

input_max_len, output_max_len

Token indices sequence length is longer than the specified maximum sequence length for this model (539 > 512). Running this sequence through the model will result in indexing errors


(1224, 103)

In [ ]:
def preprocess_function(examples):
    inputs = tokenizer(examples["dialogue"], padding="max_length", truncation=True, max_length=512)
    targets = tokenizer(examples["summary"], padding="max_length", truncation=True, max_length=150)
    inputs["labels"] = targets["input_ids"]
    return inputs

In [ ]:
train_dataset = train_data.apply(preprocess_function, axis=1)
val_dataset = validation_data.apply(preprocess_function, axis=1)

In [ ]:
# Fine Tuning

# Model
model = T5ForConditionalGeneration.from_pretrained("t5-small")

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=6,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    save_steps=500,
    eval_steps=50,
    evaluation_strategy="epoch",
)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Train the model
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: doliyaharsh (doliyaharsh-iiit-nagpur-official) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,0.427900,0.379688
2,0.376700,0.359384
3,0.374200,0.354423
4,0.361100,0.350413
5,0.348100,0.349105
6,0.351900,0.349119


TrainOutput(global_step=3000, training_loss=0.9067391357421875, metrics={'train_runtime': 1221.6146, 'train_samples_per_second': 19.646, 'train_steps_per_second': 2.456, 'total_flos': 3248203235328000.0, 'train_loss': 0.9067391357421875, 'epoch': 6.0})

In [ ]:
# Function for generating summary
device = model.device

def summarize_dialogue(dialogue):
    dialogue = clean_text(dialogue)
    inputs = tokenizer(dialogue, return_tensors="pt", truncation=True, padding="max_length", max_length=512)

    inputs = {key: value.to(device) for key, value in inputs.items()}

    outputs = model.generate(
        inputs["input_ids"],
        max_length=150,
        num_beams=4,
        early_stopping=True
    )

    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary

Test Case 1:

In [ ]:
sample_dialogue = """
Anchor: In today’s tech bulletin, a major breakthrough in artificial intelligence has just been announced by researchers at TechNova Labs.
Anchor: The new AI model, named “NeuraX,” can learn complex tasks faster than previous generations and is capable of real-time decision-making.
Scientist: This is a significant leap forward. NeuraX adapts to new environments with minimal training data, which could transform industries like healthcare, robotics, and education.
Scientist: However, with such power comes responsibility. Ethical considerations around data privacy, algorithmic bias, and automation must be addressed.
Anchor: The research team urges governments and tech companies to develop strong AI regulations. They also stress the importance of public awareness and education around AI.
Scientist: Exactly. It’s not just about developing smarter machines. It’s about ensuring these technologies are safe, transparent, and beneficial for all of society.
Anchor: With global AI conferences scheduled for later this year, experts hope the conversation will shift toward building a responsible and inclusive AI future.
"""

summary = summarize_dialogue(sample_dialogue)
print("Summary:", summary)


Summary: a breakthrough in artificial intelligence has just been announced by researchers at technova labs. neurax adapts to new environments with minimal training data, which could transform industries like healthcare, robotics, and education.


Test Case 2:

In [ ]:
sample_dialogue = """
Host: In recent developments, fine-tuning large language models has become increasingly accessible to small organizations and individual developers.
Host: Thanks to techniques like LoRA and QLoRA, the cost and compute requirements for customizing LLMs have significantly decreased.
Researcher: That’s right. These methods allow us to fine-tune only a small number of parameters, reducing memory usage and making the process more efficient.
Researcher: It’s particularly useful for domain-specific tasks, like legal document summarization or customer service chatbots.
Host: Are there any challenges developers should be aware of when fine-tuning these models?
Researcher: Definitely. Overfitting, data imbalance, and ensuring ethical outputs are all important. Plus, proper evaluation is key to make sure the model generalizes well.
Host: With advancements moving so quickly, it seems like fine-tuned LLMs might soon power most personalized AI services.
Researcher: That’s very likely. We’re entering an era where AI can be tailored for niche use cases without needing massive compute resources.
"""

summary = summarize_dialogue(sample_dialogue)
print("Summary:", summary)


Summary: lora and qlora fine-tuning large language models has become increasingly accessible to small organizations and individual developers. researcher recommends fine-tuned llms for domain-specific tasks like legal document summarization or customer service chatbots.


Test Case 3:

In [ ]:
sample_dialogue = """
Anchor: In today’s tech bulletin, a major breakthrough in artificial intelligence has just been announced by researchers at TechNova Labs.
Anchor: The new AI model, named “NeuraX,” can learn complex tasks faster than previous generations and is capable of real-time decision-making.
Scientist: This is a significant leap forward. NeuraX adapts to new environments with minimal training data, which could transform industries like healthcare, robotics, and education.
Scientist: However, with such power comes responsibility. Ethical considerations around data privacy, algorithmic bias, and automation must be addressed.
Anchor: The research team urges governments and tech companies to develop strong AI regulations. They also stress the importance of public awareness and education around AI.
Scientist: Exactly. It’s not just about developing smarter machines. It’s about ensuring these technologies are safe, transparent, and beneficial for all of society.
Anchor: With global AI conferences scheduled for later this year, experts hope the conversation will shift toward building a responsible and inclusive AI future.
"""

summary = summarize_dialogue(sample_dialogue)
print("Summary:", summary)


Summary: a breakthrough in artificial intelligence has just been announced by researchers at technova labs. neurax adapts to new environments with minimal training data, which could transform industries like healthcare, robotics, and education.
